<a href="https://colab.research.google.com/github/vanya890/AI_Study_Colab/blob/main/KR1_Kuzyk_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: Подключится к гугл диску, распокавать файлы из Neuro/KR1/ на диске.  два файла rgb-test.zip и rgb-train.zip. Распокавать в content

from google.colab import drive
import zipfile

drive.mount('/content/drive')

# Paths to the zip files on Google Drive
zip_file_paths = [
    '/content/drive/MyDrive/Neuro/KR1/rgb-test.zip',
    '/content/drive/MyDrive/Neuro/KR1/rgb-train.zip'
]

# Extract the zip files
for zip_file_path in zip_file_paths:
  try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      zip_ref.extractall('/content')
      print(f"Successfully extracted: {zip_file_path}")
  except FileNotFoundError:
    print(f"Error: File not found - {zip_file_path}")
  except zipfile.BadZipFile:
    print(f"Error: Invalid zip file - {zip_file_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Successfully extracted: /content/drive/MyDrive/Neuro/KR1/rgb-test.zip
Successfully extracted: /content/drive/MyDrive/Neuro/KR1/rgb-train.zip


In [8]:
# prompt: Построить 3 архитектуры нейронной сети. Провести классификацию (фотографии разбиты по папка внутри content/test). Добиться точности 82%. Создайте оптимизатор параметров с использованием KerasTuner. Вывести лучшую архитектуру.
!pip install keras-tuner  # Install the keras-tuner package

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import kerastuner as kt
import os

# Define image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 96

# Data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split data for validation
)

train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Define the model builder function for Keras Tuner
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
        activation='relu',
        input_shape=(img_width, img_height, 3)
    ))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

    for i in range(hp.Int('n_layers', 1, 4)):
        model.add(keras.layers.Conv2D(
            filters=hp.Int(f'conv_{i+2}_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice(f'conv_{i+2}_kernel', values=[3,5]),
            activation='relu'
        ))
        model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16), activation='relu'))
    model.add(keras.layers.Dense(len(train_generator.class_indices), activation='softmax'))

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Initialize Keras Tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Perform hyperparameter search
tuner.search(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)



Trial 11 Complete [00h 02m 11s]
val_accuracy: 0.47562500834465027

Best val_accuracy So Far: 0.48249998688697815
Total elapsed time: 00h 25m 43s

Search: Running Trial #12

Value             |Best Value So Far |Hyperparameter
112               |96                |conv_1_filter
5                 |3                 |conv_1_kernel
4                 |1                 |n_layers
80                |64                |conv_2_filter
3                 |3                 |conv_2_kernel
48                |64                |dense_1_units
0.01              |0.001             |learning_rate
80                |None              |conv_3_filter
3                 |None              |conv_3_kernel
96                |None              |conv_4_filter
5                 |None              |conv_4_kernel
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/ro

KeyboardInterrupt: 

In [9]:
# Get the best hyperparameters and build the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)

# Train the best model
model.fit(train_generator, epochs=20, validation_data=validation_generator) #Increased Epochs

# Print the summary of the best model
print(model.summary())
print(f"Best Hyperparameters: {best_hps.values}")

# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    '/content/test',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Important for consistent evaluation
)
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Epoch 1/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 50s 663ms/step - accuracy: 0.2471 - loss: 2.1007 - val_accuracy: 0.4437 - val_loss: 1.6994
Epoch 2/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 46s 638ms/step - accuracy: 0.5233 - loss: 1.4467 - val_accuracy: 0.4856 - val_loss: 1.6075
Epoch 3/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 48s 642ms/step - accuracy: 0.5881 - loss: 1.2820 - val_accuracy: 0.5225 - val_loss: 1.4903
Epoch 4/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 83s 653ms/step - accuracy: 0.6318 - loss: 1.1467 - val_accuracy: 0.5506 - val_loss: 1.4439
Epoch 5/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 46s 643ms/step - accuracy: 0.6721 - loss: 1.0135 - val_accuracy: 0.5425 - val_loss: 1.4630
Epoch 6/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 46s 635ms/step - accuracy: 0.6908 - loss: 0.9430 - val_accuracy: 0.5631 - val_loss: 1.3926
Epoch 7/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 46s 638ms/step - accuracy: 0.7349 - loss: 0.8256 - val_accuracy: 0.5831 - val_loss: 1.3426
Epoch 8/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 84s 664ms/step - accuracy: 0.7484 - loss: 0.7875 - val_accu

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)                    │ (None, 148, 148, 96)        │           2,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 74, 74, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 72, 72, 64)          │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 82944)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │       5,308,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,101,536 (61.42 MB)

 Trainable params: 5,367,178 (20.47 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 10,734,358 (40.95 MB)

None
Best Hyperparameters: {'conv_1_filter': 96, 'conv_1_kernel': 3, 'n_layers': 1, 'conv_2_filter': 64, 'conv_2_kernel': 3, 'dense_1_units': 64, 'learning_rate': 0.001, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
Found 2000 images belonging to 10 classes.
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 195ms/step - accuracy: 0.6556 - loss: 1.3302
Test Loss: 1.219653606414795, Test Accuracy: 0.7045000195503235
